In [2]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
data = pd.read_csv('file/bquxjob_157d5c5d_1941367a86e.csv')

# Display the first few rows of the DataFrame to understand its structure
data.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Campania,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2024-12-14,BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...,Google Ads,424.01,120,431,0.278422
1,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,31.16,9,27,0.333333
2,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,58.88,19,45,0.422222
3,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(BIG-PROMO)_B...,Meta,10.31,0,0,0.000000
4,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(PROMO-PEIGO-...,Meta,19.21,0,0,0.000000


In [3]:
# Convert 'Fecha' to datetime format
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Find the most recent date in the dataset
most_recent_date = data['Fecha'].max()

# Define the two periods for comparison
period_current_end = most_recent_date
period_current_start = period_current_end - pd.Timedelta(days=6)
period_previous_end = period_current_start - pd.Timedelta(days=1)
period_previous_start = period_previous_end - pd.Timedelta(days=6)

# Filter data for the two periods
df_current = data[(data['Fecha'] >= period_current_start) & (data['Fecha'] <= period_current_end)]
df_previous = data[(data['Fecha'] >= period_previous_start) & (data['Fecha'] <= period_previous_end)]

# Check the filtered data
df_current.head(), df_previous.head()

(        Fecha                                           Campania  Plataforma  \
 72 2024-12-22  BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...  Google Ads   
 73 2024-12-22  BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...        Meta   
 74 2024-12-22  BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...        Meta   
 75 2024-12-22  BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(BIG-PROMO)_B...        Meta   
 76 2024-12-22  BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(PROMO-PEIGO-...        Meta   
 
     inversion  evento_objetivo  Instalaciones       cvr  
 72     408.32               85            411  0.206813  
 73      51.71                6             18  0.333333  
 74      80.70               17             53  0.320755  
 75      10.49                0              0  0.000000  
 76      21.61                0              0  0.000000  ,
         Fecha                                           Campania  Plataforma  \
 9  2024-12-15  BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...  Google Ad

In [6]:
# Convert 'inversion' column to numeric
data['inversion'] = pd.to_numeric(data['inversion'], errors='coerce')
df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')

# Function to calculate metrics for a given period and platform
def calculate_metrics(df, platform):
    df_platform = df[df['Plataforma'] == platform]
    total_inversion = df_platform['inversion'].sum()
    total_eventos = df_platform['evento_objetivo'].sum()
    costo_medio = total_inversion / total_eventos if total_eventos != 0 else 0
    return total_inversion, costo_medio

# Get unique platforms
platforms = data['Plataforma'].unique()

# Calculate metrics for each platform in both periods
results = {
    'fecha_inicio': period_previous_start.strftime('%Y-%m-%d') + ' to ' + period_previous_end.strftime('%Y-%m-%d'),
    'fecha_fin': period_current_start.strftime('%Y-%m-%d') + ' to ' + period_current_end.strftime('%Y-%m-%d')
}
for platform in platforms:
    inv_prev, cost_prev = calculate_metrics(df_previous, platform)
    inv_curr, cost_curr = calculate_metrics(df_current, platform)
    results[platform] = {
        'inv_prev': inv_prev,
        'cost_prev': cost_prev,
        'inv_curr': inv_curr,
        'cost_curr': cost_curr,
        'cost_change_pct': ((cost_curr - cost_prev) / cost_prev * 100) if cost_prev != 0 else 0,
        'inv_change_pct': ((inv_curr - inv_prev) / inv_prev * 100) if inv_prev != 0 else 0
    }

results

C:\Users\user\AppData\Local\Temp\ipykernel_24608\2498906916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_24608\2498906916.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')


{'fecha_inicio': '2024-12-15 to 2024-12-21',
 'fecha_fin': '2024-12-22 to 2024-12-28',
 'Google Ads': {'inv_prev': np.float64(2994.09),
  'cost_prev': np.float64(4.101493150684932),
  'inv_curr': np.float64(2841.49),
  'cost_curr': np.float64(2.306404220779221),
  'cost_change_pct': np.float64(-43.766717728297046),
  'inv_change_pct': np.float64(-5.096707179810906)},
 'Meta': {'inv_prev': np.float64(1172.85),
  'cost_prev': np.float64(7.978571428571428),
  'inv_curr': np.float64(1158.55),
  'cost_curr': np.float64(6.620285714285714),
  'cost_change_pct': np.float64(-17.024171888988356),
  'inv_change_pct': np.float64(-1.2192522487956647)},
 'TikTok Ads': {'inv_prev': np.float64(1358.0),
  'cost_prev': np.float64(5.588477366255144),
  'inv_curr': np.float64(1358.0),
  'cost_curr': np.float64(3.3865336658354113),
  'cost_change_pct': np.float64(-39.40149625935162),
  'inv_change_pct': np.float64(0.0)}}

In [5]:
for plt in results:
    print(f"Platform: {plt}")
    for key, value in results[plt].items():
        print(f"{key}: {value}")
    print()

Platform: 2024-12-15 to 2024-12-21

Platform: 2024-12-22 to 2024-12-28

Platform: Google Ads
inv_prev: 2994.09
cost_prev: 4.101493150684932
inv_curr: 2841.49
cost_curr: 2.306404220779221
cost_change_pct: -43.766717728297046
inv_change_pct: -5.096707179810906

Platform: Meta
inv_prev: 1172.85
cost_prev: 7.978571428571428
inv_curr: 1158.55
cost_curr: 6.620285714285714
cost_change_pct: -17.024171888988356
inv_change_pct: -1.2192522487956647

Platform: TikTok Ads
inv_prev: 1358.0
cost_prev: 5.588477366255144
inv_curr: 1358.0
cost_curr: 3.3865336658354113
cost_change_pct: -39.40149625935162
inv_change_pct: 0.0

